In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
def scrape_web(pages):
    url = pages
    response = requests.get(url)
    chromedriver = '/Users/zhihuang/Desktop/Metis/Cars_Price_Regression_Project/chromedriver'
    os.environ['webdriver.chrome.driver'] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    
    time.sleep(3)
    
    search_box = driver.find_element_by_xpath("//input[@name='zip']")
    search_box.click()
    search_box.send_keys('10001')
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(3)
    
    name = []
    price = []
    report = []
    accident = []
    owner = []
    holder = []
    history = []
    location = []
    info = []
    
    results = int(driver.find_element_by_id('totalResultCount').text.replace(',','').split(' ')[0])
    
    if results < 1250:
        return name, price, accident, owner, holder, history, location, info
    
    time.sleep(5)
    
    while True:
        soup = BeautifulSoup(driver.page_source, 'html5lib')
    
        listings = soup.find(class_='srp-list-container--srp')
        name += listings.find_all(class_='listing-header')
        price += listings.find_all(class_='srp-list-item-price')
        report += listings.find_all(class_='title')
        location += listings.find_all(class_='srp-list-item-dealership-location')
        history += listings.find_all(class_='count')
        info += listings.find_all(class_='srp-list-item-basic-info')
        
        next_button = driver.find_element_by_xpath('//button[text()="Next"]')
        next_button.click()
        
        time.sleep(3)
        
        if len(name) > 1225:
            break
    
    size = len(name)-150
    
    for i in range(0,size):
        name[i] = name[i].text
        price[i] = price[i].text
        location[i] = location[i].text
        info[i] = info[i].text
        history[i] = history[i].get('data-count')
        
    for i in range(0,len(report),4):
        accident.append(report[i].text)
        owner.append(report[i+1].text)
        holder.append(report[i+2].text)
    
    time.sleep(1)
    driver.close()
            
    return name[:size], price[:size], accident[:size], owner[:size], holder[:size], history[:size], location[:size], info[:size]

In [3]:
import pandas as pd
import numpy as np

In [4]:
def savetocsv(raw_name, raw_price, raw_acc, raw_own, raw_hol, raw_his, raw_loc, raw_info, i):    
    if len(raw_name) > 0:
        raw_year, raw_brand, raw_model = [], [], []
        for n in raw_name:
#             print(n)
            items = n.split(' ')
            raw_year.append(int(items[0]))
            raw_brand.append(items[1])
            raw_model.append('_'.join(items[2:]))
            raw_cost = []

        for price in raw_price:
            cost = price.split(':')[1]
            cost = cost.replace('$','')
            cost = cost.replace(',','')
            raw_cost.append(cost)

        raw_city, raw_state = [], []
        for loc in raw_loc:
            loca = loc.split(':')[1]
            loca = loca.split('(')[0]
            loca = loca.split(',')
            city = loca[0]
            state = loca[1]
            raw_city.append(city)
            raw_state.append(state)

        raw_mile, raw_body, raw_engine = [], [], []
        # raw_info
        for col in raw_info:
            info = col.split(' ')
            mile = info[1].replace(',','')
            if mile == 'N/A':
                mile = 0
            raw_mile.append(mile)
            body = info[5]
            raw_body.append(body)
            engine = '_'.join(info[7:])
            raw_engine.append(engine)

        df = pd.DataFrame({'Year': raw_year,
             'Brand': raw_brand,
             'Model': raw_model,
             'Cost': raw_cost,
             'City': raw_city,
             'State': raw_state,
             'Mileage': raw_mile,
             'Accident': raw_acc,
             'Body': raw_body,
             'Engine': raw_engine,
             'Owners': raw_own,
             'Usefor':raw_hol,
             'Services':raw_his})

        df.to_csv('carfax_m'+str(i)+'.csv')

In [5]:
# url = 'https://www.carfax.com/Used-_m14'
# scrape_web(url)

In [9]:
# url = 'https://www.carfax.com/Used-Cars-in-New-York-NY_c8636'
url = 'https://www.carfax.com/Used-_m'

# raw_name, raw_price, raw_acc, raw_own, raw_hol, raw_his, raw_loc, raw_info = scrape_web(c_url)
# len(raw_name)
for i in ['1','2','3','6','8','9']:
    c_url = url+str(i)
    time.sleep(3)
    raw_name, raw_price, raw_acc, raw_own, raw_hol, raw_his, raw_loc, raw_info = scrape_web(c_url)
    
    time.sleep(3)

    savetocsv(raw_name, raw_price, raw_acc, raw_own, raw_hol, raw_his, raw_loc, raw_info, i)   